# Toronto City Neighbourhoods - part 1 + 2 
### Author: Quang Phong

### Introduction
In this 1st notebook, I will download and clean data on neighborhoods in Toronto City.

In [4]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [60]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


** Import libraries **

In [63]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium

print('Libraries imported.')

Libraries imported.


In [7]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df = df[df['Borough']!='Not assigned'][['Postal Code', 'Borough', 'Neighbourhood']].reset_index()
df = df.drop(columns='index')
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


**If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.**

In [8]:
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df.loc[df['Neighbourhood'] == 'Not assigned', 'Borough']

In [9]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [10]:
df.shape

(103, 3)

# Toronto - part 2
### Author: Quang Phong

** Read csv file of longitude and latitude **

In [15]:
df2 = pd.read_csv('D:\\Geospatial_Coordinates.csv')
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


** Merge 2 dataframes **

In [26]:
df3 = df.set_index('Postal Code').join(df2.set_index('Postal Code')).reset_index()
df3.head()


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [25]:
df3.shape

(103, 5)

# Toronto - part 3
### Author: Quang Phong

** Explore the number of boroughs in this dataframe **

In [37]:
print('The dataframe has {} boroughs.'.format(
        len(df3['Borough'].unique())
    )
)
i = 0
for val in df3['Borough'].unique():
    i += 1
    print(i, ": ", val, sep="")

The dataframe has 10 boroughs.
1: North York
2: Downtown Toronto
3: Etobicoke
4: Scarborough
5: East York
6: York
7: East Toronto
8: West Toronto
9: Central Toronto
10: Mississauga


** Create a map of area with postal code M5A and its surrounding areas, with neighbourhoods superimposed on top **

In [81]:
# create the map based on longitude and Latitude taken from dataframe 
map_M5A = folium.Map(location = [df3.loc[df3['Postal Code'] == 'M5A', 'Latitude'].item(), df3.loc[df3['Postal Code'] == 'M5A', 'Longitude'].item()], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='teal',
        fill=True,
        fill_color='teal',
        fill_opacity=0.7,
        parse_html=False).add_to(map_M5A)  
map_M5A

** Create a map of area with postal code M8Y and its surrounding areas, with neighbourhoods superimposed on top **

In [95]:
# create the map based on longitude and Latitude taken from dataframe 
map_M8Y = folium.Map(location = [df3.loc[df3['Postal Code'] == 'M8Y', 'Latitude'].item(), df3.loc[df3['Postal Code'] == 'M8Y', 'Longitude'].item()], zoom_start=12)

# add markers to map
for lat, lng, borough, pcode in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Postal Code']):
    label = '{}, {}'.format(pcode, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='teal',
        fill=True,
        fill_color='teal',
        fill_opacity=0.7,
        parse_html=False).add_to(map_M8Y)  
map_M8Y

** Define Foursquare Credentials and Version **

In [101]:

CLIENT_ID = '4HDFETJYKE1IBEUXJPPS1DLQ2FAU2KRGA5G3B3CCVBK34SFA' # your Foursquare ID
CLIENT_SECRET = 'QIWRFSWPOE31DSQ4A0O4QWKKZJNKP4IMIBOM4RYCNGX40HKR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4HDFETJYKE1IBEUXJPPS1DLQ2FAU2KRGA5G3B3CCVBK34SFA
CLIENT_SECRET:QIWRFSWPOE31DSQ4A0O4QWKKZJNKP4IMIBOM4RYCNGX40HKR


** Now, let's get the top 20 venues that are in postal code M5A within a radius of 1000 meters **

** Let's create the GET request URL **

In [113]:
LIMIT = 200
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    df3.loc[df3['Postal Code'] == 'M5A', 'Latitude'].item(), 
    df3.loc[df3['Postal Code'] == 'M5A', 'Longitude'].item(),
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=4HDFETJYKE1IBEUXJPPS1DLQ2FAU2KRGA5G3B3CCVBK34SFA&client_secret=QIWRFSWPOE31DSQ4A0O4QWKKZJNKP4IMIBOM4RYCNGX40HKR&v=20180605&ll=43.6542599,-79.3606359&radius=1000&limit=200'

** Send the GET request and examine the results **

In [114]:
results = requests.get(url).json()
results

'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '52af6dc5498e33995b0bbf03',
       'name': 'Sultan Of Samosas',
       'location': {'address': '1 Oak Street',
        'lat': 43.66022749787444,
        'lng': -79.36622571571063,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.66022749787444,
          'lng': -79.36622571571063}],
        'distance': 802,
        'cc': 'CA',
        'city': 'Toronto',
        'state': 'ON',
        'country': 'Canada',
        'formattedAddress': ['1 Oak Street', 'Toronto ON', 'Canada']},
       'categories': [{'id': '4bf58dd8d48988d10f941735',
         'name': 'Indian Restaurant',
         'pluralName': 'Indian Restaurants',
         'shortName': 'Indian',
         'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/indian_',
          'suffix': '.png'},
         'primary': True}],
       'photos': {'count': 0, 'groups': []}},
      'referralId': 'e-0-52af6dc5498e33995b0bbf03-83'},
     {'reasons': {'cou

In [115]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [116]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Impact Kitchen,Restaurant,43.656369,-79.356980
3,The Distillery Historic District,Historic Site,43.650244,-79.359323
4,Distillery Sunday Market,Farmers Market,43.650075,-79.361832


** How many venues were returned by Foursquare? **

In [122]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


** Let's create a function to repeat the same process to all the postal codes in Toronto **

In [135]:
def getNearbyVenues(pcodes, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for pcode, lat, lng in zip(pcodes, latitudes, longitudes):
        print(pcode)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            pcode, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [136]:
toronto_venues = getNearbyVenues(pcodes=df3['Postal Code'], latitudes=df3['Latitude'], longitudes=df3['Longitude']
                                  )

M3A
M4A
M5A
M6A
M7A
M9A
M1B
M3B
M4B
M5B
M6B
M9B
M1C
M3C
M4C
M5C
M6C
M9C
M1E
M4E
M5E
M6E
M1G
M4G
M5G
M6G
M1H
M2H
M3H
M4H
M5H
M6H
M1J
M2J
M3J
M4J
M5J
M6J
M1K
M2K
M3K
M4K
M5K
M6K
M1L
M2L
M3L
M4L
M5L
M6L
M9L
M1M
M2M
M3M
M4M
M5M
M6M
M9M
M1N
M2N
M3N
M4N
M5N
M6N
M9N
M1P
M2P
M4P
M5P
M6P
M9P
M1R
M2R
M4R
M5R
M6R
M7R
M9R
M1S
M4S
M5S
M6S
M1T
M4T
M5T
M1V
M4V
M5V
M8V
M9V
M1W
M4W
M5W
M8W
M9W
M1X
M4X
M5X
M8X
M4Y
M7Y
M8Y
M8Z


In [137]:
print(toronto_venues.shape)
toronto_venues.head()

(4985, 7)


,Postal Code,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,M3A,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
2,M3A,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café
3,M3A,43.753259,-79.329656,A&W,43.760643,-79.326865,Fast Food Restaurant
4,M3A,43.753259,-79.329656,Bruno's valu-mart,43.746143,-79.324630,Grocery Store


** Let's check how many venues were returned for each postal code **

In [190]:
toronto_venues.groupby('Postal Code').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postal Code,,,,,,
M1B,19,19,19,19,19,19
M1C,5,5,5,5,5,5
M1E,24,24,24,24,24,24
M1G,9,9,9,9,9,9
M1H,30,30,30,30,30,30
M1J,16,16,16,16,16,16
M1K,32,32,32,32,32,32
M1L,29,29,29,29,29,29
M1M,12,12,12,12,12,12


** Let's find out how many unique categories can be curated from all the returned venues **

In [191]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 337 uniques categories.


** Now we will examine the top 10 top common venues in each postal code **

In [192]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add postal code column back to dataframe
toronto_onehot['Postal Code'] = toronto_venues['Postal Code'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Postal Code,Accessories Store,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsagent,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skat

In [193]:
toronto_onehot.shape

(4985, 338)

In [184]:
# Next, let's group rows by postal code and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Postal Code').mean().reset_index()
toronto_grouped

,Postal Code,Accessories Store,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsagent,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skat

In [194]:
# Let's confirm the new size
toronto_grouped.shape

(103, 338)

In [195]:
# Let's print each postal code along with the top 10 most common venues
num_top_venues = 10

for hood in toronto_grouped['Postal Code']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Postal Code'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

              Pizza Place  0.04
4                       Café  0.04
5                        Gym  0.03
6  Middle Eastern Restaurant  0.03
7               Dessert Shop  0.03
8                 Restaurant  0.03
9          Indian Restaurant  0.03


----M4T----
                venue  freq
0       Grocery Store  0.08
1         Coffee Shop  0.08
2  Italian Restaurant  0.06
3     Thai Restaurant  0.05
4                Park  0.05
5                 Gym  0.05
6          Restaurant  0.05
7      Sandwich Place  0.03
8                Bank  0.03
9    Sushi Restaurant  0.03


----M4V----
                venue  freq
0         Coffee Shop  0.10
1    Sushi Restaurant  0.06
2     Thai Restaurant  0.05
3  Italian Restaurant  0.05
4       Grocery Store  0.04
5                Park  0.04
6          Restaurant  0.04
7      Sandwich Place  0.03
8                Café  0.03
9   French Restaurant  0.03


----M4W----
                 venue  freq
0          Coffee Shop  0.13
1                 Park  0.13
2        Groc

** Put the previous results into a pandas dataframe **

In [196]:
# let's write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [197]:

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postal Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
pcodes_venues_sorted = pd.DataFrame(columns=columns)
pcodes_venues_sorted['Postal Code'] = toronto_grouped['Postal Code']

for ind in np.arange(toronto_grouped.shape[0]):
    pcodes_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Cluster Labels,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,M1B,Fast Food Restaurant,Coffee Shop,Trail,Bank,Caribbean Restaurant,Bakery,Paper / Office Supplies Store,Gym,Park,Supermarket
1,4,M1C,Breakfast Spot,Park,Burger Joint,Italian Restaurant,Playground,Zoo,Filipino Restaurant,Event Space,Falafel Restaurant,Farm
2,0,M1E,Pizza Place,Bank,Coffee Shop,Fast Food Restaurant,Bus Line,Sandwich Place,Greek Restaurant,Grocery Store,Discount Store,Fried Chicken Joint
3,4,M1G,Park,Coffee Shop,Dog Run,Mobile Phone Shop,Fast Food Restaurant,Chinese Restaurant,Indian Restaurant,Dessert Shop,Dim Sum Restaurant,Falafel Restaurant
4,0,M1H,Indian Restaurant,Pizza Place,Bakery,Gas Station,Bank,Coffee Shop,Pharmacy,Burger Joint,Yoga Studio,Bus Line
5,0,M1J,Ice Cream Shop,Convenience Store,Restaurant,Garden,Grocery Store,Sandwich Place,Coffee Shop,Fast Food Restaurant,Train Station,Japanese Restaurant
6,0,M1K,Chinese Restaurant,Coffee Shop,Burger Joint,Discount Store,Pizza Place,Fast Food Restaurant,Grocery Store,Bus Line,Spa,Caribbean Restaurant
7,0,M1L,Coffee Shop,Pharmacy,Bus Line,Bakery,Intersection,Grocery Store,Bank,Sandwich Place,General Entertainment,Beer Store
8,0,M1M,Pizza Place,Ice Cream Shop,Beach,Hardware Store,Restaurant,Park,Chinese Restaurant,Sports Bar,Hookah Bar,Farm
9,4,M1N,Park,Convenience Store,Auto Workshop,Gym Pool,Gym,General Entertainment,Motorcycle Shop,Diner,Restaurant,College Stadium


** Run k-means to cluster the postal code into 5 clusters **

In [170]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Postal Code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 4, 0, 4, 0, 0, 0, 0, 0, 4])

In [189]:
# add clustering labels
pcodes_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df3

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(pcodes_venues_sorted.set_index('Postal Code'), on='Postal Code')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0,Park,Bus Stop,Convenience Store,Shopping Mall,Pharmacy,Food & Drink Shop,Coffee Shop,Fast Food Restaurant,Café,Skating Rink
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,Coffee Shop,Sporting Goods Shop,Pizza Place,French Restaurant,Boxing Gym,Café,Men's Store,Park,Grocery Store,Gym / Fitness Center
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Pub,Theater,Café,Park,Bakery,Restaurant,Diner,Breakfast Spot,Italian Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0,Clothing Store,Fast Food Restaurant,Coffee Shop,Vietnamese Restaurant,Restaurant,Fried Chicken Joint,Furniture / Home Store,Dessert Shop,Sushi Restaurant,Accessories Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Café,Park,Sushi Restaurant,Ramen Restaurant,Thai Restaurant,Italian Restaurant,Cosmetics Shop,Clothing Store,Gastropub


In [176]:
# let's visualize the resulting clusters
# create map
map_clusters = folium.Map(location=[43.651070, -79.347015], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postal Code'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

** Cluster 1 **

In [178]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,Park,Bus Stop,Convenience Store,Shopping Mall,Pharmacy,Food & Drink Shop,Coffee Shop,Fast Food Restaurant,Café,Skating Rink
1,North York,0,Coffee Shop,Sporting Goods Shop,Pizza Place,French Restaurant,Boxing Gym,Café,Men's Store,Park,Grocery Store,Gym / Fitness Center
2,Downtown Toronto,0,Coffee Shop,Pub,Theater,Café,Park,Bakery,Restaurant,Diner,Breakfast Spot,Italian Restaurant
3,North York,0,Clothing Store,Fast Food Restaurant,Coffee Shop,Vietnamese Restaurant,Restaurant,Fried Chicken Joint,Furniture / Home Store,Dessert Shop,Sushi Restaurant,Accessories Store
4,Downtown Toronto,0,Coffee Shop,Café,Park,Sushi Restaurant,Ramen Restaurant,Thai Restaurant,Italian Restaurant,Cosmetics Shop,Clothing Store,Gastropub
6,Scarborough,0,Fast Food Restaurant,Coffee Shop,Trail,Bank,Caribbean Restaurant,Bakery,Paper / Office Supplies Store,Gym,Park,Supermarket
7,North York,0,Coffee Shop,Japanese Restaurant,Pizza Place,Burger Joint,Indian Restaurant,Café,Tennis Court,Bar,Liquor Store,Supermarket
8,East York,0,Coffee Shop,Brewery,Pizza Place,Construction & Landscaping,Gym / Fitness Center,Athletics & Sports,Soccer Stadium,Breakfast Spot,Fast Food Restaurant,Gastropub
9,Downtown Toronto,0,Coffee Shop,Gastropub,Restaurant,Japanese Restaurant,Hotel,Diner,Italian Restaurant,Creperie,Cosmetics Shop,New American Restaurant
10,North York,0,Grocery Store,Fast Food Restaurant,Pizza Place,Coffee Shop,Gas Station,Italian Restaurant,BBQ Joint,Gym Pool,Latin American Restaurant,Mediterranean Restaurant


** Cluster 2 **

In [179]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,Etobicoke,1,Lounge,Coffee Shop,Zoo,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


** Cluster 3 **

In [180]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
95,Scarborough,2,Gay Bar,Filipino Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field


** Cluster 4 **

In [181]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,North York,3,Park,Pool,Filipino Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


** Cluster 5 **

In [182]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Etobicoke,4,Pharmacy,Park,Café,Baseball Field,Bakery,Grocery Store,Shopping Mall,Skating Rink,Bank,Golf Course
11,Etobicoke,4,Park,Pizza Place,Bank,Hotel,Fish & Chips Shop,Restaurant,Grocery Store,Gym,Mexican Restaurant,Clothing Store
12,Scarborough,4,Breakfast Spot,Park,Burger Joint,Italian Restaurant,Playground,Zoo,Filipino Restaurant,Event Space,Falafel Restaurant,Farm
22,Scarborough,4,Park,Coffee Shop,Dog Run,Mobile Phone Shop,Fast Food Restaurant,Chinese Restaurant,Indian Restaurant,Dessert Shop,Dim Sum Restaurant,Falafel Restaurant
46,North York,4,Park,Grocery Store,Shopping Mall,Vietnamese Restaurant,Pizza Place,Bank,Farmers Market,Elementary School,Escape Room,Ethiopian Restaurant
49,North York,4,Park,Coffee Shop,Bakery,Pizza Place,Athletics & Sports,Mediterranean Restaurant,Chinese Restaurant,Gas Station,Convenience Store,Dim Sum Restaurant
57,North York,4,Park,Storage Facility,Bakery,Golf Course,Auto Workshop,Gas Station,Discount Store,African Restaurant,Convenience Store,Flea Market
58,Scarborough,4,Park,Convenience Store,Auto Workshop,Gym Pool,Gym,General Entertainment,Motorcycle Shop,Diner,Restaurant,College Stadium
66,North York,4,Park,Restaurant,Coffee Shop,Dog Run,Gym,Business Service,Convenience Store,Intersection,Playground,French Restaurant
101,Etobicoke,4,Park,Ice Cream Shop,Shopping Mall,Gym / Fitness Center,Eastern European Restaurant,Italian Restaurant,Bus Stop,Baseball Field,Fast Food Restaurant,Ethiopian Restaurant
